In this lab, you will create, train, evaluate, and make predictions on a model using [Apache Beam](https://beam.apache.org/) and [TensorFlow](https://www.tensorflow.org/). In particular, you will train a model to predict the molecular energy based on the number of carbon, hydrogen, oxygen, and nitrogen atoms.

This lab is marked as optional because you will not be interacting with Beam-based systems directly in future exercises. Other courses of this specialization also use tools that abstract this layer. Nonetheless, it would be good to be familiar with it since it is used under the hood by TFX which is the main ML pipelines framework that you will use in other labs. Seeing how these systems work will let you explore other codebases that use this tool more freely and even make contributions or bug fixes as you see fit. If you don't know the basics of Beam yet, we encourage you to look at the [Minimal Word Count example here](https://beam.apache.org/get-started/wordcount-example/) for a quick start and use the [Beam Programming Guide](https://beam.apache.org/documentation/programming-guide) to look up concepts if needed.

The entire pipeline can be divided into four phases:
 1. Data extraction
 2. Preprocessing the data
 3. Training the model
 4. Doing predictions

You will focus particularly on Phase 2 (Preprocessing) and a bit of Phase 4 (Predictions) because these use Beam in its implementation.

In [ ]:
# NOTE: Some of the packages used in this lab are not compatible with the
# default Python version in Colab (Python3.8 as of January 2023).
# The commands below will setup the environment to use Python3.7 instead.
# Please *DO NOT* restart the runtime after running these.

# Install packages needed to downgrade to Python3.7
!apt-get install python3.7 python3.7-distutils python3-pip

# Configure the Colab environment to use Python3.7 by default
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 100

In [ ]:
# Download the scripts
!wget https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/raw/main/course4/week2-ungraded-labs/C4_W2_Lab_4_ETL_Beam/data/molecules.tar.gz

# Unzip the archive
!tar -xvzf molecules.tar.gz --one-top-level

In [ ]:
# Download the scripts
!wget https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/raw/main/course4/week2-ungraded-labs/C4_W2_Lab_4_ETL_Beam/data/molecules.tar.gz

# Unzip the archive
!tar -xvzf molecules.tar.gz --one-top-level

In [ ]:
# Define working directory
WORK_DIR = "results"

The `data-extractor.py` file extracts and decompresses the specified SDF files. In later steps, the example preprocesses these files and uses the data to train and evaluate the machine learning model. The file extracts the SDF files from the public source and stores them in a subdirectory inside the specified working directory.

In [ ]:
# Print the help documentation. You can ignore references to GCP because you will be running everything in Colab.
!python ./molecules/data-extractor.py --help

In [ ]:
# Run the data extractor
!python ./molecules/data-extractor.py --max-data-files 1 --work-dir={WORK_DIR}

In [ ]:
# List working directory
!ls {WORK_DIR}

In [ ]:
# Print one record
!sed '/$$$$/q' {WORK_DIR}/data/00000001_00025000.sdf

## Phase 2: Preprocessing

The next script: `preprocess.py` uses an Apache Beam pipeline to preprocess the data. The pipeline performs the following preprocessing actions:

1. Reads and parses the extracted SDF files.
2. Counts the number of different atoms in each of the molecules in the files.
3. Normalizes the counts to values between 0 and 1 using tf.Transform.
4. Partitions the dataset into a training dataset and an evaluation dataset.
5. Writes the two datasets as TFRecord objects.

Apache Beam transforms can efficiently manipulate single elements at a time, but transforms that require a full pass of the dataset cannot easily be done with only Apache Beam and are better done using [tf.Transform](https://www.tensorflow.org/tfx/guide/tft). Because of this, the code uses Apache Beam transforms to read and format the molecules, and to count the atoms in each molecule. The code then uses `tf.Transform` to find the global minimum and maximum counts in order to normalize the data.
